In [252]:
import io
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pylab as plt
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [254]:
movie_titles = ['movieId', 'title']
titles = pd.read_csv('C:/Users/cajethan/Desktop/INFO_MGT/thesis_info_mgt/datasets/online25/movies.csv', encoding = "Latin1", usecols = movie_titles)
titles.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [255]:
titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 2 columns):
movieId    2500 non-null int64
title      2500 non-null object
dtypes: int64(1), object(1)
memory usage: 39.2+ KB


In [256]:
column_rating = ['userId', 'movieId', 'rating']
rating_score = pd.read_csv('C:/Users/cajethan/Desktop/INFO_MGT/thesis_info_mgt/datasets/online25/ratings.csv', usecols = column_rating)
#rating_score['movieId'] = rating_score['movieId'].astype('str')
rating_score.head()

,userId,movieId,rating
0,12882,1,4.0
1,12882,32,3.5
2,12882,47,5.0
3,12882,50,5.0
4,12882,110,4.5


In [257]:
rating_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264505 entries, 0 to 264504
Data columns (total 3 columns):
userId     264505 non-null int64
movieId    264505 non-null int64
rating     264505 non-null float64
dtypes: float64(1), int64(2)
memory usage: 6.1 MB


In [258]:
merged_data = pd.merge(rating_score, titles, on = 'movieId')
merged_data.head()

,userId,movieId,rating,title
0,12882,1,4.0,Toy Story (1995)
1,121987,1,4.5,Toy Story (1995)
2,18127,1,4.0,Toy Story (1995)
3,17318,1,4.0,Toy Story (1995)
4,69519,1,4.0,Toy Story (1995)


In [259]:
def assign_to_testset(df):
    sampled_ids = np.random.choice(df.index,size=np.int64(np.ceil(df.index.size * 0.2)),replace=False)
    df.loc[sampled_ids, 'for_testing'] = True
    return df

merged_data['for_testing'] = False
grouped = merged_data.groupby('userId', group_keys=False).apply(assign_to_testset)
train_data = merged_data[grouped.for_testing == False]
test_data = merged_data[grouped.for_testing == True]
print(train_data.shape)
print(test_data.shape)
print(train_data.index & test_data.index)

print("Training data_set has "+ str(train_data.shape[0]) +" ratings")
print("Test data set has "+ str(test_data.shape[0]) +" ratings")

(211263, 5)
(53242, 5)
Int64Index([], dtype='int64')
Training data_set has 211263 ratings
Test data set has 53242 ratings


In [260]:
#compute the mean 
mean_rating = train_data.groupby(by = 'userId', as_index = False)['rating'].mean()
#find the average 
avg_rating = pd.merge(train_data, mean_rating, on = 'userId')
avg_rating.head()

,userId,movieId,rating_x,title,for_testing,rating_y
0,121987,1,4.5,Toy Story (1995),False,4.159722
1,121987,588,4.5,Aladdin (1992),False,4.159722
2,121987,1148,4.0,Wallace & Gromit: The Wrong Trousers (1993),False,4.159722
3,121987,1196,4.5,Star Wars: Episode V - The Empire Strikes Back...,False,4.159722
4,121987,1198,4.5,Raiders of the Lost Ark (Indiana Jones and the...,False,4.159722


In [294]:
#rename rating_x and rating_y
avg_rating = avg_rating.rename(columns = {"rating_x": "rating", "rating_y": "mean_rating"})
avg_rating = avg_rating.drop(columns = "for_testing")
avg_rating.head()

,userId,movieId,rating,title,mean_rating,base_rating
0,121987,1,4.5,Toy Story (1995),4.159722,0.340278
1,121987,588,4.5,Aladdin (1992),4.159722,0.340278
2,121987,1148,4.0,Wallace & Gromit: The Wrong Trousers (1993),4.159722,-0.159722
3,121987,1196,4.5,Star Wars: Episode V - The Empire Strikes Back...,4.159722,0.340278
4,121987,1198,4.5,Raiders of the Lost Ark (Indiana Jones and the...,4.159722,0.340278


In [295]:
#now we get the adjusted rating
avg_rating['base_rating'] = avg_rating['rating'] - avg_rating['mean_rating']
avg_rating.head()

,userId,movieId,rating,title,mean_rating,base_rating
0,121987,1,4.5,Toy Story (1995),4.159722,0.340278
1,121987,588,4.5,Aladdin (1992),4.159722,0.340278
2,121987,1148,4.0,Wallace & Gromit: The Wrong Trousers (1993),4.159722,-0.159722
3,121987,1196,4.5,Star Wars: Episode V - The Empire Strikes Back...,4.159722,0.340278
4,121987,1198,4.5,Raiders of the Lost Ark (Indiana Jones and the...,4.159722,0.340278


In [296]:
avg_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211263 entries, 0 to 211262
Data columns (total 6 columns):
userId         211263 non-null int64
movieId        211263 non-null int64
rating         211263 non-null float64
title          211263 non-null object
mean_rating    211263 non-null float64
base_rating    211263 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 11.3+ MB


In [297]:
#make a pivot_table out of the dataframe
df_final = pd.pivot_table(avg_rating, values = 'base_rating', index = ['userId'], columns = ['movieId'])
df_final.head()

movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,-0.805825,NaN,NaN,NaN,NaN,NaN,-1.305825,NaN,-0.805825,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,1.322823,NaN,NaN,NaN,NaN,1.322823,NaN,NaN,0.322823,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,0.713415,0.213415,NaN,NaN,NaN,1.213415,NaN,NaN,NaN,NaN,...,-1.286585,NaN,NaN,NaN,NaN,NaN,-0.786585,NaN,NaN,-0.786585
910,1.126617,0.126617,-0.373383,NaN,-0.373383,-0.373383,NaN,NaN,NaN,0.126617,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.626617,NaN,NaN


In [298]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 862 entries, 316 to 138200
Columns: 2500 entries, 1 to 112852
dtypes: float64(2500)
memory usage: 16.4 MB


In [299]:
#replace all the NaN values with the movie average in each row
df_final = df_final.fillna(df_final.mean(axis = 0))
df_final.head()

movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,-0.805825,-0.426157,-0.531687,-0.626957,-0.501798,0.361617,-1.305825,-0.696137,-0.805825,-0.099362,...,0.072626,-0.039814,0.276082,0.267125,0.452155,0.263754,0.240251,0.251296,0.363814,0.398912
320,0.198402,-0.426157,-0.531687,-0.626957,-0.501798,0.361617,-0.136981,-0.696137,-0.137494,-0.099362,...,0.072626,-0.039814,0.276082,0.267125,0.452155,0.263754,0.240251,0.251296,0.363814,0.398912
359,1.322823,-0.426157,-0.531687,-0.626957,-0.501798,1.322823,-0.136981,-0.696137,0.322823,-0.099362,...,0.072626,-0.039814,0.276082,0.267125,0.452155,0.263754,0.240251,0.251296,0.363814,0.398912
370,0.713415,0.213415,-0.531687,-0.626957,-0.501798,1.213415,-0.136981,-0.696137,-0.137494,-0.099362,...,-1.286585,-0.039814,0.276082,0.267125,0.452155,0.263754,-0.786585,0.251296,0.363814,-0.786585
910,1.126617,0.126617,-0.373383,-0.626957,-0.373383,-0.373383,-0.136981,-0.696137,-0.137494,0.126617,...,0.072626,-0.039814,0.276082,0.267125,0.452155,0.263754,0.240251,0.626617,0.363814,0.398912


In [300]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 862 entries, 316 to 138200
Columns: 2500 entries, 1 to 112852
dtypes: float64(2500)
memory usage: 16.4 MB


In [308]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

#calculate the similarity using cosine similarity 
cos_sim = cosine_similarity(df_final)
np.fill_diagonal(cos_sim, 0)
#create an empty dataframe to store all the similar movies 
sim_movies = pd.DataFrame(cos_sim, index = df_final.index, columns = df_final.index)
sim_movies.head()

userId,316,320,359,370,910,975,1015,1387,1447,1588,...,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
userId,,,,,,,,,,,,,,,,,,,,,
316,0.000000,0.940857,0.711595,0.714869,0.758680,0.916437,0.830003,0.882631,0.959256,0.754294,...,0.871512,0.918794,0.902369,0.943352,0.938322,0.947474,0.627177,0.717698,0.964276,0.788666
320,0.940857,0.000000,0.726149,0.724937,0.757159,0.930092,0.839407,0.898877,0.975724,0.764860,...,0.889475,0.928028,0.922288,0.956072,0.944936,0.959576,0.649090,0.724939,0.979187,0.808503
359,0.711595,0.726149,0.000000,0.566919,0.592281,0.696596,0.654837,0.676851,0.739027,0.590391,...,0.670259,0.694460,0.691846,0.721414,0.722551,0.725377,0.456800,0.543256,0.740805,0.590787
370,0.714869,0.724937,0.566919,0.000000,0.598890,0.708114,0.647382,0.675370,0.744135,0.599359,...,0.681410,0.711453,0.703891,0.726555,0.724315,0.732184,0.450687,0.550140,0.744912,0.592958
910,0.758680,0.757159,0.592281,0.598890,0.000000,0.746249,0.675066,0.700461,0.776023,0.600858,...,0.697480,0.732479,0.732906,0.765991,0.758737,0.765543,0.483788,0.575893,0.780017,0.628543


In [269]:
# #next, we compute the k-nearest neighbours.
# #how do we determine the value of K??????
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn import metrics
# from sklearn.model_selection import train_test_split
# from sklearn import preprocessing

# # x = avg_rating.drop(['base_rating'], axis = 1)
# # y = avg_rating['base_rating']

# # print(x.head())
# # print(y.head())

# X = merged_data.drop(['rating', 'title'], axis = 1)
# y = merged_data['rating']
# # X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))

# print(X.head())
# print(y.head())

# X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2)

In [270]:
# error = []
# for i in range (1, 40):
#     nearest = KNeighborsClassifier(n_neighbors = i)
#     nearest.fit(X_train, y_train)
#     pred_i = nearest.predict(X_test)
#     error.append(np.mean(pred_i != y_test))
    
# plt.figure(figsize = (10, 6))
# plt.plot(range (1, 40), error, color = 'blue', linestyle = 'dashed', 
#          marker = 'o',markerfacecolor = 'red', markersize = 10)
# plt.title('Error Rate vs. K Value')
# plt.xlabel('K')
# plt.ylabel('Error Rate')
# print("Minimum error:-", min(error),"at K =",error.index(min(error)))

In [309]:
def find_neighbors(df, n):
    sort = np.argsort(df.values, axis = 1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending = False)
            .iloc[:n].index,
            index = ['top{}'.format(i) for i in range(1, n + 1)]), axis = 1)
    return df

In [310]:
#top n similar neighbors for each user
similar_users = find_neighbors(sim_movies, 30)
similar_users.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,
316,100240,138176,96936,88932,104732,51460,121403,5268,104529,13927,...,129869,21401,84598,80585,124981,121987,27142,22338,12721,118304
320,138176,96936,121403,42944,88932,1447,51460,125012,51255,104529,...,21401,121987,93203,27142,104732,12721,124981,118304,94333,22338
359,138176,84598,1447,72633,96936,42944,100240,51255,65255,124981,...,104732,114335,75001,136037,93203,21401,22338,125012,27142,24802
370,86309,44194,94333,138176,129869,1447,96936,24802,88455,72633,...,12930,93203,125012,54114,5268,100240,124454,121987,38256,124981
910,96936,138176,27142,104732,51460,100240,121987,107991,88932,124454,...,1447,126292,118304,121403,42944,114335,124981,13927,21401,51255


In [311]:
#next, we return all the similar movies
def similarities(user1, user2):
    common_movies = avg_rating[avg_rating.userId == user1].merge(avg_rating[avg_rating.userId == user2]
                    ,on = 'movieId'
                    ,how = "inner")
    return common_movies

In [312]:
sim_test = similarities(630, 10002)
sim_test = sim_test.loc[:, ['rating_x', 'rating_y', 'title']].rename(columns = {"rating_x": "user1_rating", "rating_y": "user2_rating"})
sim_test.head()

C:\Users\cajethan\Anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,user1_rating,user2_rating,title


In [313]:
#now we predict the rating a given user is most like to make for a given movie
# def predict_rating(user, item):
#     person = similar_users[similar_users.index == user].values.squeeze().tolist()
#     movie = df_final.loc[:, item][index.isin(person).notnull()]
#     avg_user = mean_rating.loc[mean_rating['userId'] == user, 'rating'].values[0]
#     index = movie.index.values.squeeze().tolist()
#     corr = sim_movies.loc[user, index]
#     combine = pd.concat([movie, corr],axis = 1)
#     combine.columns = ['score', 'correlation']
#     combine['score'] = combine.apply(lambda x:x['score'] * x['correlation'], axis = 1)
#     numerator = combine['score'].sum()
#     denominator = combine['correlation'].sum()
#     predicted_rating = avg_user + (numerator/denominator)
#     return predicted_rating

def User_item_score(user,item):
    #.The values() method returns a view object that displays a list of all the values in the dictionary. 
    a = similar_users[similar_users.index==user].values
    #squeeze() removes single-dimensional entries - it removes the user's similarity with itself
    #tolist() makes a list of values (userid's  in this instance)
    b = a.squeeze().tolist()
    
    c = df_final.loc[:, item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    # get the average rating of the target user
    avg_user = mean_rating.loc[mean_rating['userId'] == user,'rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = sim_movies.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

In [314]:
# #predict the rating for user 20 on movie 120
# prediction = predict_rating(10, 120)
# print("the predicted rating for this movie is ", prediction)


score = User_item_score(380, 7371)
print("score (u,i) is",score)

IndexError: index 0 is out of bounds for axis 0 with size 0